In [23]:
import pandas
import csv
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
UserID_ArtistMBID_ArtistName_mappings_filtered_filePath = '/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/Data pre-processing-3/UserID_ArtistMBID_ArtistName_mappings_filtered.csv'

In [4]:
UserID_ArtistMBID_ArtistName_DF = pandas.read_csv(UserID_ArtistMBID_ArtistName_mappings_filtered_filePath)

In [5]:
UserID_ArtistMBID_ArtistName_DF.head

<bound method NDFrame.head of           user_id                           artist_mbid       artist_name
0           17240  edef3cfa-4e5e-4d64-8bd8-20f9dc1d8cad  Tiken Jah Fakoly
1           16930  4341463c-3bc5-483a-bcff-d32e4650c9b6           Fraktus
2           15136  85a55a8c-c042-40d6-9dc5-fb85d0d926cf             Népal
3            1626  77c167d2-4965-4421-830a-9815e4956475      Dinosaur Jr.
4           15276  44223a23-a89a-4815-9eab-e36ce9d0ebae     Rappin’ 4‐Tay
...           ...                                   ...               ...
37462637    11632  c1b0fe0a-779d-43ed-b193-4370f0d0f88f             結束バンド
37462638    21135  1fa3bded-908f-47be-9784-3c9ef850682b       Oliver Tree
37462639     8208  a94f3763-2ee2-4cc4-a176-978dc8bf96e3     Jasper Tygner
37462640    12850  25ca48d2-9017-4a4f-8594-a6d8892b8a13     Igneous Flame
37462641    16299  05b75ee5-98cd-430e-b1ec-ec0f61fdabda         Desiigner

[37462642 rows x 3 columns]>

In [6]:
'''
The pandas.DataFrame.groupby() method is used to group rows of data based on one or more columns of a DataFrame.
It returns a GroupBy object which has some useful methods to aggregate, transform, or filter the data within each group.
In my case I use it to perform the count of listenings for each (unique) user_id regarding  an artistID/name
'''
dataFrame = UserID_ArtistMBID_ArtistName_DF.groupby(['user_id','artist_mbid','artist_name']).size().reset_index(name='count')

In [7]:
dataFrame.head(15)

,user_id,artist_mbid,artist_name,count
0,1,000fc734-b7e1-4a01-92d1-f544261b43f5,Cocteau Twins,1
1,1,0019749d-ee29-4a5f-ab17-6bfa11deb969,DJ Food,1
2,1,0039c7ae-e1a7-4a7d-9b49-0cbc716821a6,Death Cab for Cutie,3
3,1,00950dec-8f3a-4a17-9717-e7872a954d8b,Terranova,1
4,1,01252145-c9e8-4de5-a480-9b2bed05450a,Émilie Simon,1
5,1,0145f13d-4c4d-43b5-a7fb-cd69da8ca202,Natalie Walker,1
6,1,0171668b-ba7a-4cef-8a28-22aa782dd56c,Zoë Keating,2
7,1,01726f9b-6e54-48e5-af05-871678ce7d1c,Руки вверх!,1
8,1,01f5eb45-b422-4f30-9c95-a1639bafe33b,Pilote,2
9,1,01f91d2c-e729-46eb-aa4a-4da3ccc64e56,Dirk Freymuth,1


In [10]:
'''
Strings are a nice way to visualize the artist names to the user interface,
but they are not great to make mappings and calculations.
pandas.DataFrame.astype('category') can help
'''
dataFrame['artist_name'] = dataFrame['artist_name'].astype('category')

# compute lists of codes and categories, will be useful when we have to map between code/cat
# get the categorical codes as an array
artistNames_codes = dataFrame['artist_name'].cat.codes.values
print(artistNames_codes)

# get the categories as an array
artistNames_cat = dataFrame['artist_name'].cat.categories.values
print(artistNames_cat)

[ 45055  51175  57808 ... 106182 275659  44938]
['!!!' '!Action Pact!' '!Attention!' ... '🌴1am バイブBlunts' '👁\u200d🗨📲' '🧫']


In [11]:
# substitute the artist_name strings with artist_name cat codes
dataFrame['artist_name'] = dataFrame['artist_name'].cat.codes
dataFrame.head()

,user_id,artist_mbid,artist_name,count
0,1,000fc734-b7e1-4a01-92d1-f544261b43f5,45055,1
1,1,0019749d-ee29-4a5f-ab17-6bfa11deb969,51175,1
2,1,0039c7ae-e1a7-4a7d-9b49-0cbc716821a6,57808,3
3,1,00950dec-8f3a-4a17-9717-e7872a954d8b,219438,1
4,1,01252145-c9e8-4de5-a480-9b2bed05450a,264897,1


In [12]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 32.8 MB/s eta 0:00:00


In [13]:
from implicit.nearest_neighbours import bm25_weight
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix

/usr/local/lib/python3.8/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [20]:
# Convert your DataFrame to a CSR sparse matrix (similar to coordinate format)
# the coordinate format is a compact and compressed way to represent a sparse matrix
# (since in a sparse matrix most of the values do not exist, it is useful to compute only the existing mappings)
sparse_matrix = csr_matrix((dataFrame['count'], (dataFrame['user_id'], dataFrame['artist_name'])))
# print(sparse_matrix)

In [21]:
from implicit.nearest_neighbours import bm25_weight

# weight the matrix, both to reduce impact of users that have played the same artist thousands of times
# and to reduce the weight given to popular items
sparse_matrix_weighted = bm25_weight(sparse_matrix, K1=100, B=0.8)

In [22]:
# model = AlternatingLeastSquares(factors=20, regularization=0.01)
# model.fit(sparse_matrix_weighted)

/usr/local/lib/python3.8/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.04721403121948242 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

In [25]:
# with open('/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/Data pre-processing-3/Implicit_Model.p', 'wb') as implicitModelPickleFile:
#  pickle.dump(model, implicitModelPickleFile)

In [26]:
with open('/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Audio and Music processing lab - assignment/Task 1/Data pre-processing-3/Implicit_Model.p', 'rb') as handle:
    implicitModel = pickle.load(handle)

In [84]:
# get recommendations for user 1
user_id = 100
'''
The .recommend call will compute the N best recommendations for each user in the input,
and return the itemids in the ids array as well as the computed scores in the scores array.
We can see what the musicians are recommended for each user by looking up the ids in the artists array
'''
artist_name_codes , scores = implicitModel.recommend(user_id, sparse_matrix[user_id], filter_already_liked_items=False)
original_names = artistNames_cat[artist_name_codes]
pandas.DataFrame({"artist": original_names, "score": scores})

,artist,score
0,Tiësto,0.065418
1,Scooter,0.063196
2,Faithless,0.062839
3,The Prodigy,0.061715
4,Armin van Buuren,0.061060
5,Lady Gaga,0.058975
6,The Chemical Brothers,0.058823
7,Madonna,0.057824
8,Robyn,0.057668
9,Kylie Minogue,0.056642


In [85]:
artist_ToFindASimilarArtistFor = artist_name_codes[6]
artistNames_cat[artist_ToFindASimilarArtistFor]

'The Chemical Brothers'

In [86]:
# find related artists
similar_artists, similarity_scores = implicitModel.similar_items(artist_ToFindASimilarArtistFor)

similar_artists = artistNames_cat[similar_artists]
# print(similar_artists)
# print(similarity_scores)

pandas.DataFrame({"artist": similar_artists, "score": similarity_scores})

,artist,score
0,The Chemical Brothers,1.000000
1,Fatboy Slim,0.972132
2,The Prodigy,0.962766
3,Massive Attack,0.952227
4,Röyksopp,0.951154
5,Moby,0.947169
6,Underworld,0.944241
7,Depeche Mode,0.938132
8,Orbital,0.935547
9,Faithless,0.935032
